https://developer.riotgames.com/

<b>Import pacakges<b>

In [168]:
import requests
import numpy as np
import csv
import time
import pandas as pd

<b>Declare variables<b>

In [169]:
#these variables are used as inputs for api calls.
key_and_region = {
    'key' : '',
    'region' : '',
    'globalRegion' : ''
}

given_user = {
    'name' : '',
    'puuid' : ''
}


<b>Define functions<b>

In [170]:
#initial setup of the api key and regions.
def key_and_region_setup():

    with open("initial_key_and_region.txt", "r") as f:
        values = f.readlines()
        key_and_region['key'] = values[0].strip()
        key_and_region['region'] = values[1].strip()
        key_and_region['globalRegion'] = values[2].strip()

In [171]:
#initinal setup of user with developer's data.
def user_setup():

    with open("initial_user.txt", "r") as f:
        values = f.readlines()
        given_user['name'] = values[0].strip()
        given_user['puuid'] = values[1].strip()

In [172]:
#change user variables to another user.
def user_change(user_name, user_puuid):
    
    given = {
        'name' : user_name,
        'puuid' : user_puuid
    }
    
    given_user.update({'name' : given['name'], 'puuid' : given['puuid']})

In [173]:
#Using user's name, get puuid of the user.
def get_puuid_by_name(user_name=given_user['name']):
    
    given = {
        'name' : user_name
    }
    
    url = F"{key_and_region['region']}/lol/summoner/v4/summoners/by-name/{given['name']}?api_key={key_and_region['key']}"
    data = requests.get(url).json() #.json() changes the format to json and organize output vertically.
    
    given_user.update({'name' : data['name'],'puuid' : data['puuid']})
    

    return given_user

In [174]:
#Using user's puuid, get user's name.
def get_name_by_puuid(user_puuid=given_user['puuid']):
    
    given = {
        'puuid' : user_puuid
    }
    
    url = F"{key_and_region['region']}/lol/summoner/v4/summoners/by-puuid/{given['puuid']}?api_key={key_and_region['key']}"
    data = requests.get(url).json() 
    
    given_user.update({'name' : data['name'],'puuid' : data['puuid']})
    

    return given_user

In [175]:
#Append data of given_user to user_data.csv.
#data is appended with \n at the end. If you add data by hand, type enter at the end to keep the format.
def add_given_user_to_user_data():
    
    with open("user_data.csv", "a") as f:
        appender = csv.writer(f)
        appender.writerow([given_user['name'], given_user['puuid']])

In [176]:
#Remove duplicated data from user_data.csv.
#if you have opened the file, cursor doesn't matter, but typing enter matters since it is recorded as \n. BE AWARE!
#if you delete data and practice it again, note that \n can still remain at the end if you don't drag the entire line.
def remove_duplicated_of_user_data():
    before_removal = open("user_data.csv" , 'r')
  
    before_separation = before_removal.readlines()
    first_row = before_separation[0]
    users = before_separation[1:]
    users = list(set(users))
    users.sort()
    
    after_removal = open("user_data.csv", 'w')
    after_removal.write(first_row)
    for user in users:
        after_removal.write(user)
        
    before_removal.close()
    after_removal.close()

In [177]:
#using user's puuid, get records of recent aram match IDs of the user.
#input must be between 0 and 100
def get_recent_aram_matchId_list_by_puuid(list_size=20):
    
    given = {
        'size' : list_size
    }
    
    url = F"{key_and_region['globalRegion']}/lol/match/v5/matches/by-puuid/{given_user['puuid']}/ids?queue=450&start=0&count={given['size']}&api_key={key_and_region['key']}"
    data = requests.get(url).json()
    
    return data

In [178]:
#using match ID, get detailed data of the match.
def get_matchData_by_matchId(match_ID):
    
    given = {
        'ID' : match_ID
    }
    
    url = F"{key_and_region['globalRegion']}/lol/match/v5/matches/{given['ID']}?api_key={key_and_region['key']}"
    data = requests.get(url).json()
    
    return data

In [179]:
#Show how many match IDs have been collected.
def how_many_matchIds():
    matchIds = np.load('match_id_data_before_extraction.npy')
    print("we have ", len(matchIds), " matchIDs.")

In [180]:
#Show how many puuids have been collected.
def how_many_puuids():        
    user_list_file = open("user_data.csv" , 'r')
    user_list = user_list_file.readlines()
    users = user_list[1:]
    print("we have " , len(users), " users' puuids.")
    user_list_file.close()

In [181]:
#Show how many games' detailed data have been collected.
def how_many_game_data():
    matchIds = np.load('match_id_data_after_extraction.npy', allow_pickle = True)
    print("we have ", len(matchIds), " matchIDs.")

<b>Define functions using the functions above<b>

In [182]:
#Collect more matchIds.
#Iterate user_data.csv, collect their recent 100 aram records, remove the overlapped records, and save it.
#1 cycle goes for first 100 users of user_data.csv. 
#The least efficient function for now, as it repeatedly goes for first several users of the list,
#if given cycle is not large enough. But since we don't know which player has gotten new records the most,
#cannot figure out how to improve this function's productivity.
def collect_more_matchIds(cycle = 10):
    
    collected_matchIds = np.array([])
    one_to_hundred = 0
    hundreds = 0
    
    user_list_file = open("user_data.csv" , 'r')
    user_list = user_list_file.readlines()
    first_row = user_list[0]
    users = user_list[1:]
    
    for user in users:
        user_change(user.split(',')[0], user.split(',')[1].strip())
        matchIds = np.array(get_recent_aram_matchId_list_by_puuid(100)) #calls api
        collected_matchIds = np.concatenate((collected_matchIds, matchIds)) # need to concatenate in form of tuple = ((a,b))
        one_to_hundred += 1
        
        if one_to_hundred == 100:
                hundreds += 1
                print(hundreds, " cycle has been done.")
                one_to_hundred = 0
                if hundreds == cycle:
                    break
                time.sleep(125)
        
    existing_matchIds = np.load('match_id_data_before_extraction.npy')
    added_matchIds = np.concatenate((existing_matchIds, collected_matchIds))
    added_matchIds = np.unique(added_matchIds)
    np.save('match_id_data_before_extraction', added_matchIds)
    user_list_file.close()
    how_many_matchIds()
    print("The entire process has been done as ", cycle, " cycles were set. Finish the program.")

In [183]:
#Collect more puuids.
#Iterate match_id_data_before_extraction.npy, collect data of each game's participants, and save it.
#When iterating, match ID is checked if it already has been through this process. If so, the match ID is passed without calling api.
#1 cycles goes for first 100 match IDs of match_id_data_before_extraction.npy.
#100 calls per 2 min. Idk why but 10 participants are passed, not 9(I thought 11 calls per 1 outer loop, but it is somehow 10)
def collect_more_puuids(cycle = 10):
    
    before_extraction = np.load('match_id_data_before_extraction.npy')
    after_extraction = np.load('match_id_data_after_extraction.npy', allow_pickle=True)
    one_to_hundred = 0
    hundreds = 0
    loop_breaker = False
    
    for matchId in before_extraction:
        
        if loop_breaker == True:
            break
        
        if not matchId in after_extraction:
            participants = get_matchData_by_matchId(matchId)['metadata']['participants'] #calls api
        
            for participant in participants:
                get_name_by_puuid(participant) #calls api
                add_given_user_to_user_data()
                one_to_hundred += 1    
            
                if one_to_hundred == 100:
                    hundreds += 1
                    print(hundreds, " cycle has been done.")
                    one_to_hundred = 0
                    
                    if hundreds == cycle:
                        loop_breaker = True
                        break
                    
                    time.sleep(125)
    
    remove_duplicated_of_user_data()
    how_many_puuids()
    print("The entire process has been done as ", cycle, " cycles were set. Finish the program.")

In [184]:
#Collect more game data.
#Iterate match_id_data_before_extraction.npy, collect data(played champions, kda, victory or defeat) of each game, and save it.
#When iterating, match ID is checked if it already has been through this process. If so, the match ID is passed without calling api.
#After process, newly processed match ID is added to match_id_data_after_extraction.
#1 cycles goes for first 100 match IDs of match_id_data_before_extraction.npy.
def collect_more_game_data(cycle = 10):
    
    before_extraction = np.load('match_id_data_before_extraction.npy')
    after_extraction = np.load('match_id_data_after_extraction.npy', allow_pickle=True)
    recently_extracted = np.array([])
    one_to_hundred = 0
    hundreds = 0
    
    game_data = pd.read_csv("game_data.csv", header=0)
    
    for matchId in before_extraction:
        
        if not matchId in after_extraction:
            recently_extracted = np.append(recently_extracted, matchId)
            game_data = get_matchData_by_matchId(matchId) #calls api
            players = game_data['info']['participants']
            
            with open("game_data.csv", "a") as f:
                appender = csv.writer(f)
                for player in players:
                    appender.writerow([game_data['metadata']['matchId'], player['championName'], player['kills'],
                                 player['deaths'], player['assists'], player['win']])
            one_to_hundred += 1
            
            if one_to_hundred == 100:
                hundreds += 1
                print(hundreds, " cycle has been done.")
                one_to_hundred = 0
                
                if hundreds == cycle:
                    collected_matchIds = np.concatenate((after_extraction, recently_extracted)) 
                    np.save('match_id_data_after_extraction', collected_matchIds)
                    how_many_game_data()
                    print("The entire process has been done as ", cycle, " cycles were set. Finish the program.")
                    break
                    
                time.sleep(125)

<b>Setup at the beginning<b>

In [166]:
key_and_region_setup()
user_setup()

In [167]:
how_many_matchIds()
how_many_puuids()
how_many_game_data()

we have  318436  matchIDs.
we have  3700  users' puuids.
we have  9400  matchIDs.
